In [1]:
# Example SQLite dataset downloaded externally --BONUS for demonstrating system flexibility with undetermined schemas
!python kaggle_database_downloader.py "gastonsaracusti/model-car-mint-classics"

100%|███████████████████████████████████████| 26.0k/26.0k [00:00<00:00, 371kB/s]
Extracting files...

Dataset downloaded to:
"/home/mohamed/.cache/kagglehub/datasets/gastonsaracusti/model-car-mint-classics/versions/2"

Dataset moved to:
"/home/mohamed/Projects/llm-for-sql-queries/data/gastonsaracusti/model-car-mint-classics"

Database File:
"/home/mohamed/Projects/llm-for-sql-queries/data/gastonsaracusti/model-car-mint-classics/data.sqlite"

Process exited with status code: 0


In [2]:
# Database creator through the template given by ASAP Systems for Barcloud
!python asap_database_creator.py "aayman/asap-database-template" "./script.sql"

Database created successfully!

Database File:
/home/mohamed/Projects/llm-for-sql-queries/data/aayman/asap-database-template/data.sqlite


In [3]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import sqlite3
import keyring
from openai import OpenAI
from string import Template
import os
import logging
from time import time
import re
import json

DB_FILEPATH = '/home/mohamed/Projects/llm-for-sql-queries/data/aayman/asap-database-template/data.sqlite'
LLM_TASK_TEMPLATE_FILEPATH = os.path.abspath('./llm_task_template.md')
MODEL_NAME = 'gpt-4o-mini'
PROVIDER = 'openai'

class Helper:

    def __new__(self):
        raise NotImplementedError('This class is meant to be static, it cannot be instantiated.')
    
    @staticmethod
    def read_task_template(filepath):
        try:
            file = open(filepath)
            return file.read()
        except FileNotFoundError:
            message = 'LLM task description template not found!'
            logging.exception(message)
            print(f'{message}\nKindly check the error logs for traceback details.')
        finally:
            file.close()


class Timer:
    
    def __enter__(self):
        self._start = self._time_ms()
        return self
    
    def __exit__(self, *args):
        self._elapsed = self._time_ms() - self._start

    def _time_ms(self):
        return round(1e3 * time())
    
    @property
    def start(self):
        return self._start
    
    @property
    def elapsed(self):
        return self._elapsed


class OpenAIChatBot:
    
    LLM_TASK_TEMPLATE = Helper.read_task_template(LLM_TASK_TEMPLATE_FILEPATH)
    
    def __init__(self, api_key, model, db_filepath):
        self._client    = OpenAI(api_key=api_key)
        self._model     = model
        self.db_filepath = db_filepath
    
    @property
    def client(self):
        return self._client
    
    @property
    def model(self):
        return self._model
    
    @property
    def db_filepath(self):
        return self._db_filepath
    
    @db_filepath.setter
    def db_filepath(self, value: str):
        self._db_filepath = value
        self.db_schema   = self._extract_db_schema(value)
    
    @property
    def db_schema(self):
        return self._db_schema
    
    @db_schema.setter
    def db_schema(self, value: str):
        self._db_schema = value
        self._chat_history = [
            {'role': 'system', 'content': Template(self.LLM_TASK_TEMPLATE).substitute({'db_schema': value}) }
        ]

    @property
    def chat_history(self):
        return self._chat_history
    
    def send(self, user_query):
        self._chat_history.append({'role': 'user', 'content': user_query})
        response_details = {'provider': PROVIDER}
        try:
            with Timer() as t:
                response = self._client.chat.completions.create(model=self._model, messages=self._chat_history)
            self._chat_history.append({'role': 'system', 'content': response.choices[0].message.content})
            response_details = self._extract_response_details(response)
            response_details.update({
                'latency_ms': t.elapsed,
                'status': 'ok',
            })
        except:
            logging.exception('Fatal Error.\nKindly check the error logs for traceback details.')
            response_details.update({
                'model': self._model,
                'status': 'error',
            })
        return response_details
    
    @staticmethod
    def _extract_response_details(response):
        details = json.loads(re.sub(r'```(json)?', '', response.choices[0].message.content))
        details.update({
            'token_usage': {
                'prompt_tokens': response.usage.prompt_tokens,
                'completion_tokens': response.usage.completion_tokens,
                'total_tokens': response.usage.total_tokens,
            },
            'model': response.model,
        })
        return details
    
    @staticmethod
    def _extract_db_schema(db_filepath: str):
        try:
            conn = sqlite3.connect(DB_FILEPATH)
            cursor = conn.cursor()
            cursor.execute('SELECT sql FROM sqlite_master WHERE type="table";')
            rows = cursor.fetchall()
        finally:
            conn.close()
        create_table_cmds = list(zip(*rows))[0]
        db_schema = '\n'.join(create_table_cmds)
        return db_schema

In [4]:
chatbot = OpenAIChatBot(
    api_key    = keyring.get_password('openai', 'default'),
    model      = MODEL_NAME,
    db_filepath = DB_FILEPATH,
)

response_details = chatbot.send('What is the total billed amount for the last quarter?')

In [5]:
response_details

{'Answer': 'The total billed amount for the last quarter has been calculated.',
 'SQL': "SELECT SUM(TotalAmount) AS TotalBilledAmount\nFROM Bills\nWHERE BillDate >= DATE('now', 'start of quarter', '-3 months') AND BillDate < DATE('now', 'start of quarter');",
 'token_usage': {'prompt_tokens': 1624,
  'completion_tokens': 74,
  'total_tokens': 1698},
 'model': 'gpt-4o-mini-2024-07-18',
 'latency_ms': 1792,
 'status': 'ok'}

In [8]:
response_details = chatbot.send('Which vendor supplied the most assets?')
response_details

{'Answer': 'Here are the vendors sorted by the number of assets they supplied.',
 'SQL': "SELECT v.VendorName, COUNT(a.AssetId) AS AssetCount\nFROM Vendors v\nJOIN Assets a ON v.VendorId = a.VendorId\nWHERE a.Status <> 'Disposed'\nGROUP BY v.VendorId, v.VendorName\nORDER BY AssetCount DESC;",
 'token_usage': {'prompt_tokens': 1713,
  'completion_tokens': 88,
  'total_tokens': 1801},
 'model': 'gpt-4o-mini-2024-07-18',
 'latency_ms': 2098,
 'status': 'ok'}

In [9]:
chatbot.chat_history

[{'role': 'system',
  'content': 'TASK:\n------\nYou are a SQLite transliterator. \\\nGiven the following SQLite SCHEMA, I\'d \\\nlike you to transliterate any subsequent \\\nuser query (given in natural language) \\\nabout the dataset into an SQLite query \\\nfor fetching the answer from its database.\n\n---\n\nSCHEMA:\n--------\nCREATE TABLE Customers (\n    CustomerId INTEGER PRIMARY KEY AUTOINCREMENT,\n    CustomerCode VARCHAR(50) UNIQUE NOT NULL,\n    CustomerName TEXT NOT NULL,\n    Email TEXT NULL,\n    Phone TEXT NULL,\n    BillingAddress1 TEXT NULL,\n    BillingCity TEXT NULL,\n    BillingCountry TEXT NULL,\n    CreatedAt DATETIME NOT NULL DEFAULT (datetime(\'now\')),\n    UpdatedAt DATETIME NULL,\n    IsActive BOOLEAN NOT NULL DEFAULT 1\n)\nCREATE TABLE sqlite_sequence(name,seq)\nCREATE TABLE Vendors (\n    VendorId INTEGER PRIMARY KEY AUTOINCREMENT,\n    VendorCode VARCHAR(50) UNIQUE NOT NULL,\n    VendorName TEXT NOT NULL,\n    Email TEXT NULL,\n    Phone TEXT NULL,\n    Ad

In [6]:
# from http.server import HTTPServer, BaseHTTPRequestHandler
# import json
# from urllib.parse import urlparse, parse_qs

# class Server(BaseHTTPRequestHandler):
#     def __init__(self, db_filepath):